# Compulsory Assignment 3 Teststrecke

Leonard Brenk, Finn Federsan, Felix Wltschek

## Question 1

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
# read csvs

true_csv = pd.read_csv("True.csv")
fake_csv = pd.read_csv("Fake.csv")
fake_csv['label'] = 0
true_csv['label'] = 1
df = pd.concat([fake_csv, true_csv], ignore_index=True)

In [ ]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


### 1.1. Preprocess

In [ ]:
# Combine title and text, drop unused columns, clean punctuation, extra spaces, and make lowercase

df['text'] = df['title'] + ' ' + df['text'] # merge
df = df.drop(['title', 'subject', 'date'], axis=1)
df['text'] = df['text'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", " ", x)) # replace punctuation
df['text'] = df['text'].apply(lambda x: re.sub("\s+", " ", x)) # replace multiple spaces with one
df['text'] = df['text'].apply(lambda x: x.lower())


### 1.2 Stoppwords

In [ ]:
# Remove stopwords and short words from the 'text' column in the DataFrame

stop_words = set(stopwords.words('english'))
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words and len(word) > 2]))

### 1.3 Split

In [ ]:
# Split data

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

### 1.4 Tokenize

In [ ]:
# Tokenize the text data in train and test sets, pad sequences to a fixed length, and determine the vocabulary size

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=250, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=250, padding='post')

size = len(tokenizer.word_index) + 1

In [ ]:
type(X_train_seq)

list

### 1.5 Model 

In [ ]:
# Bi-LSTM model with embedding layer, two bidirectional LSTM layers, dense layers with Adam optimizer, binary cross-entropy loss, and accuracy metric

model = Sequential() # for adding the layers 
model.add(Embedding(size, 128, input_length=125)) # turns tokens into layers of vectors 
model.add(Bidirectional(LSTM(64, return_sequences=True))) # bidirectional to learn from both future and past 
model.add(Bidirectional(LSTM(64))) 
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



### 1.6 Training

In [ ]:

model.fit(X_train_padded, y_train, batch_size=64, epochs=2, validation_split=0.2)

Epoch 1/2


2023-04-07 14:52:46.042679: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


ValueError: in user code:

    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File

### 1.7 Evaluate accuracy

In [ ]:
# Evaluate the model on test data, print the accuracy, and return the loss and accuracy values

loss, accuracy = model.evaluate(X_test_padded, y_test)
print("accuracy: " + str(accuracy))
print("loss: " + str(loss))

ValueError: in user code:

    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "/Users/leonardbrenk/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "